In [27]:
!pip uninstall -y transformers tokenizers peft accelerate datasets huggingface_hub torchvision

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: tokenizers 0.22.1
Uninstalling tokenizers-0.22.1:
  Successfully uninstalled tokenizers-0.22.1
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: huggingface-hub 0.36.0
Uninstalling huggingface-hub-0.36.0:
  Successfully uninstalled huggingface-hub-0.36.0
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126


In [2]:
!pip install --upgrade transformers==4.45.0 huggingface_hub datasets

  Using cached huggingface_hub-1.1.5-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0


In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
import torch
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import argparse
import warnings
warnings.filterwarnings("ignore")


In [5]:
import os
print(os.listdir())

['.config', 'semeval', '.ipynb_checkpoints', '=4.45.0', '=0.20.0', '=0.13.0', 'sample_data']


In [6]:
ds = load_dataset("DaniilOr/SemEval-2026-Task13", "A")

README.md:   0%|          | 0.00/801 [00:00<?, ?B/s]

task_a/task_a_training_set_1.parquet:   0%|          | 0.00/203M [00:00<?, ?B/s]

task_a/task_a_validation_set.parquet:   0%|          | 0.00/40.5M [00:00<?, ?B/s]

task_a/task_a_test_set_sample.parquet:   0%|          | 0.00/593k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/500000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['code', 'generator', 'label', 'language'],
        num_rows: 500000
    })
    validation: Dataset({
        features: ['code', 'generator', 'label', 'language'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['code', 'generator', 'label', 'language'],
        num_rows: 1000
    })
})

We suggest using a single class, it will make refinement easier.

In your implementation, feel free to update the training procedure, change model and do whatever feels right

In [2]:
class CodeBERTTrainer:
    def __init__(self, max_length=512, model_name="microsoft/codebert-base"):
        self.max_length = max_length
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.num_labels = None

    def load_and_prepare_data(self):

        try:
            print("Loading dataset from HuggingFace Hub...")
        ds = load_dataset("DaniilOr/SemEval-2026-Task13", "A")

        train_df = ds["train"].to_pandas()
        val_df   = ds["validation"].to_pandas()

        print(f"Train samples: {len(train_df)}, Validation samples: {len(val_df)}")

        # Prüfe Spalten
        if 'code' not in train_df.columns or 'label' not in train_df.columns:
            raise ValueError("Dataset must contain 'code' and 'label' columns")

        train_df = train_df.dropna(subset=['code', 'label'])
        val_df   = val_df.dropna(subset=['code', 'label'])

        train_df['label'] = train_df['label'].astype(int)
        val_df['label']   = val_df['label'].astype(int)

        self.num_labels = train_df['label'].nunique()

        print(f"Number of labels: {self.num_labels}")
        print(train_df['label'].value_counts().sort_index())

        return train_df, val_df

        except Exception as e:
            print(f"Error loading dataset: {e}")
            raise

    def initialize_model_and_tokenizer(self):
        print(f"Initializing {self.model_name} model and tokenizer...")

        self.tokenizer = RobertaTokenizer.from_pretrained(self.model_name)

        self.model = RobertaForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_labels,
            problem_type="single_label_classification"
        ).to('cuda')

        print(f"Model initialized with {self.num_labels} labels")

    def tokenize_function(self, examples):
        return self.tokenizer(
            examples['code'],
            truncation=True,
            padding=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

    def prepare_datasets(self, train_df, val_df):
        print("Preparing datasets for training...")

        train_dataset = Dataset.from_pandas(train_df[['code', 'label']])
        val_dataset = Dataset.from_pandas(val_df[['code', 'label']])

        train_dataset = train_dataset.map(
            self.tokenize_function,
            batched=True,
            remove_columns=['code']
        )
        val_dataset = val_dataset.map(
            self.tokenize_function,
            batched=True,
            remove_columns=['code']
        )

        train_dataset = train_dataset.rename_column('label', 'labels')
        val_dataset = val_dataset.rename_column('label', 'labels')

        return train_dataset, val_dataset

    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        accuracy = accuracy_score(labels, predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

        return {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }

    def train(self, train_dataset, val_dataset, output_dir="./results", num_epochs=3, batch_size=16, learning_rate=2e-5):
        print("Starting training...")
        print(self.model)
        print(self.model.device)
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            # warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=5,
            evaluation_strategy="steps",
            eval_steps=500,
            save_strategy="steps",
            save_steps=500,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            remove_unused_columns=False,
            learning_rate=learning_rate,
            lr_scheduler_type="linear",
            save_total_limit=2,
            report_to=[],
        )

        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)

        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )
        print(f"Start training")
        trainer.train()

        trainer.save_model()
        self.tokenizer.save_pretrained(output_dir)

        print(f"Training completed. Model saved to {output_dir}")

        return trainer

    def evaluate_model(self, trainer, val_dataset):
        print("Evaluating model...")

        predictions = trainer.predict(val_dataset)
        y_pred = np.argmax(predictions.predictions, axis=1)
        y_true = predictions.label_ids

        print("Classification Report:")
        print(classification_report(y_true, y_pred))

        return predictions

    def run_full_pipeline(self, output_dir="./results", num_epochs=3, batch_size=16, learning_rate=2e-5):
        try:
            train_df, val_df = self.load_and_prepare_data()

            self.initialize_model_and_tokenizer()

            train_dataset, val_dataset = self.prepare_datasets(train_df, val_df)

            trainer = self.train(
                train_dataset, val_dataset,
                output_dir=output_dir,
                num_epochs=num_epochs,
                batch_size=batch_size,
                learning_rate=learning_rate
            )

            self.evaluate_model(trainer, val_dataset)

            print("Pipeline completed successfully!")
            return trainer

        except Exception as e:
            print(f"Error in pipeline: {e}")
            raise


In [3]:
OUTPUT_DIR = "taskA-model"

trainer_obj = CodeBERTTrainer(
    max_length=256,
)

t = trainer_obj.run_full_pipeline(
    output_dir=OUTPUT_DIR,
    num_epochs=10,
    batch_size=16,
    learning_rate=2e-5
)


Error loading dataset: [Errno 2] No such file or directory: '/kaggle/input/sem-eval-2026-task-13-subtask-a/Task_A/train.parquet'
Error in pipeline: [Errno 2] No such file or directory: '/kaggle/input/sem-eval-2026-task-13-subtask-a/Task_A/train.parquet'


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/sem-eval-2026-task-13-subtask-a/Task_A/train.parquet'

In [ ]:
import torch
import logging
from itertools import chain
from datasets import load_dataset
from tqdm import tqdm


@torch.no_grad()
def predict_with_trainer(trainer_obj, parquet_path, output_path, max_length=512, batch_size=16, device=None):
    """
    Uses trainer_obj.model and trainer_obj.tokenizer to run streaming inference
    over a parquet file with columns ['ID','code'] and writes 'ID,prediction' CSV.
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Pull model & tokenizer from your trainer object
    model = trainer_obj.model
    tokenizer = trainer_obj.tokenizer if hasattr(trainer_obj, "tokenizer") else trainer_obj.args._setup_devices and None
    if tokenizer is None and hasattr(trainer_obj, "tokenizer"):
        tokenizer = trainer_obj.tokenizer
    if tokenizer is None:
        raise ValueError("trainer_obj must have a tokenizer (e.g., provided when creating the Trainer).")

    model.to(device)
    model.eval()

    # Stream parquet (no RAM blowup)
    ds = load_dataset("parquet", data_files=parquet_path, split="train", streaming=True)

    # Validate schema and re-chain the first row back into the stream
    it = iter(ds)
    first = next(it)
    if not {"ID", "code"}.issubset(first.keys()):
        raise ValueError("Parquet file must contain 'ID' and 'code' columns")
    stream = chain([first], it)

    def batcher(iterator, bs):
        buf = []
        for ex in iterator:
            buf.append(ex)
            if len(buf) == bs:
                yield buf
                buf = []
        if buf:
            yield buf

    with open(output_path, "w") as f:
        f.write("ID,prediction\n")

        for batch in tqdm(batcher(stream, batch_size), desc="Predicting"):
            codes = [row["code"] for row in batch]
            ids   = [row["ID"] for row in batch]

            enc = tokenizer(
                codes,
                truncation=True,
                padding=True,
                max_length=max_length,
                return_tensors="pt",
            )
            input_ids = enc["input_ids"].to(device)
            attention_mask = enc["attention_mask"].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
            pred_labels = logits.argmax(dim=-1).cpu().tolist()

            for ex_id, pred in zip(ids, pred_labels):
                f.write(f"{ex_id},{pred}\n")

    print(f"Predictions saved to {output_path}")


In [ ]:
# After training:
# trainer_obj = CodeBERTTrainer(...).run_full_pipeline(output_dir=..., ...)

TEST_PARQUET = "/kaggle/input/sem-eval-2026-task-13-subtask-a/Task_A/test.parquet"  # adjust if needed
OUT_CSV = "/kaggle/working/submission.csv"

predict_with_trainer(
    trainer_obj=t,
    parquet_path=TEST_PARQUET,
    output_path=OUT_CSV,
    max_length=256,
    batch_size=32,
    device="cuda"
)

print("Wrote:", OUT_CSV)
